In [7]:
import sys
import os
import numpy as np
import pandas as pd
import string
import json
import pickle
from zipfile import ZipFile
import requests
from io import BytesIO
import re

In [3]:
# Get maps dataframe back from the pickle file
maps_df = pd.read_pickle("../Data_Gather_Filter_Download/downloaded_maps_df_small.pkl")

In [ ]:
def count_notes(notes, song_length):
    counts = [0] * (song_length / 10)
    for note in notes:
        note_time = (note['_time'] / bpm) * 60
        note_bucket = int(note_time / 10)
        counts[note_bucket] += 1
    return counts

In [ ]:
def compare_model(maps_df, max_comparisons):
    num_comparisons = 0
    for file_path in maps_df.file_path.unique():
        if num_comparisons > max_comparisons:
            break
        if file_path != "NOT_FOUND":
            with ZipFile("../Data_Gather_Filter_Download/{}".format(file_path)) as folder:
                folder_name = file_path.rsplit('/', 1)[-1]
                song_length = (maps_df.loc[maps_df['file_path'] == file_path])['length'].values[0]
                bpm = (maps_df.loc[maps_df['file_path'] == file_path])['bpm'].values[0]
                folder.extractall(path="./Comparisons/{}".format(folder_name))
                filenames = folder.namelist()
                difficulties_dats = [diff for diff in filenames if diff in ["Expert.dat", "ExpertPlus.dat"]]
                for difficulties_dat in difficulties_dats:
                    with folder.open(difficulties_dat) as diff_dat:
                        dat_json = json.load(diff_dat)
                        counts = count_notes(dat_json['_notes'], song_length)
                    
                    

    